In [22]:
# In[ ]:

import pandas as pd
import dateutil.parser

stock=pd.read_csv('data/stock/삼성전자2010.csv')
stock=stock.set_index('Date')
stock.index=pd.to_datetime(stock.index.map(dateutil.parser.parse)+pd.DateOffset(hours=15,minutes=30))

#결측값 제거
stock = stock[stock.index.dayofweek!=6]#이거 필요한가?
stock = stock.dropna(how='any',axis=0)
stock['Change'] = stock['Close'].astype('float').pct_change()

stock=stock[['Change']]
stock=stock.sort_values('Date')

#이동평균 적용
stock=stock.rolling('3d').mean()

news=pd.read_csv('data/news/삼성전자_2010.01.01_2021.04.12_3.csv')

#temp code(remove whitespaces)
#news=news.dropna()
#news['time']=news['time'].replace(u'(입력 :)|(수정 :.*)|(\xa0)|\n|\t','',regex=True).str.strip(' ')

news['time']=pd.to_datetime(news['time'].map(dateutil.parser.parse))
news=news.sort_values('time')

labeled_data = pd.merge_asof(news,stock,left_on='time',right_on='Date',direction='forward')

#labeled_data['text']=labeled_data['text'].str.split('.')
#labeled_data=labeled_data.explode('text')

labeled_data=labeled_data.drop(columns=['Unnamed: 0'])
labeled_data=labeled_data.rename(columns={'Change':'label'})

labeled_data.head()
# labeled_data.dropna().to_csv('data/labeled/samsung_2010_2021.csv')
# %%


,title,url,text,date,time,label
0,삼성전자 ‘서프라이즈’ 계속 됐다,https://m.news.naver.com/read.nhn?mode=LSD&mid...,원본보기 삼성전자가 5년 만에 연결기준 ‘연간 영업이익 10조원 시대’를 다시 열었...,2010.01.01,2010-01-01 05:06:02,NaN
1,세계 전자산업 향연 `CES' 7일 개막,https://m.news.naver.com/read.nhn?mode=LSD&mid...,"원본보기삼성.LG, 3DTV 등 신제품 대거 출품 (서울=연합뉴스) 김종수 기...",2010.01.01,2010-01-01 08:01:03,NaN
2,삼성電 19.7%로 압도적 1위,https://m.news.naver.com/read.nhn?mode=LSD&mid...,[머니투데이 정영일 기자][[2010년 증시는? 전문가 223명 설문] 물려주고 싶...,2010.01.01,2010-01-01 10:02:02,NaN
3,[토요기획] KT 전창진 감독의 새해 소망,https://m.news.naver.com/read.nhn?mode=LSD&mid...,"금정산 호랑이 '두마리 토끼'를 잡아라 ""요즘 사직체육관에 관중들이 많이 오십니다...",2010.01.02,2010-01-02 09:33:01,NaN
4,"새해 증시개장 첫날, 유망 추천주는?",https://m.news.naver.com/read.nhn?mode=LSD&mid...,"동양종합금융증권은 2일 새해 증시 연초 유망종목으로 삼성전기와 대한항공, 삼영이엔씨...",2010.01.02,2010-01-02 10:21:01,NaN


In [4]:
import pandas as pd
import numpy as np
import re

news = pd.read_csv('data/labeled/samsung_2010_2021.csv')
news.drop(['Unnamed: 0'], axis=1, inplace=True)

news.head()

,title,url,text,date,time,label
0,최지성 사장 “비전2020 구체화해 삼성전자 100년 기업으로”,https://m.news.naver.com/read.nhn?mode=LSD&mid...,“올해 100년 기업을 향한 ‘비전2020’을 구체화하기 위해 초경쟁력을 확보하...,2010.01.04,2010-01-04 17:03:02,0.016069
1,"[CES 2010]삼성전자, 친환경 LED 대거 선보여",https://m.news.naver.com/read.nhn?mode=LSD&mid...,원본보기[아시아경제 김정민 기자]삼성전자가 7일 미국 라스베이거스에서 열리는 세계 ...,2010.01.05,2010-01-05 11:00:00,0.016069
2,"삼성전자 ""MP3플레이어도 보는 시대""",https://m.news.naver.com/read.nhn?mode=LSD&mid...,원본보기 (서울=연합뉴스) 조성흠 기자 = 삼성전자는 오는 7일 미국 라스베이...,2010.01.05,2010-01-05 11:00:08,0.016069
3,"김연아-오서 코치, 삼성전자 에어컨 모델 발탁",https://n.news.naver.com/sports/general/articl...,[마이데일리 = 김용우 기자] '피겨 퀸' 김연아(19·고려대)와 브라이언 오서 코...,2010.01.05,2010-01-05 12:08:01,0.016069
4,삼성전자 사상 최고가…83만원 돌파,https://m.news.naver.com/read.nhn?mode=LSD&mid...,삼성전자(005930)가 개장과 함께 83만원을 넘어서며 사상최고가를 경신했다. 6...,2010.01.06,2010-01-06 10:01:01,0.019592


In [9]:
idx = 0
newss = news.copy()
for txt in newss['text']:
    txt = re.sub(r'[^\w]', ' ', txt) + ''
    txt = txt.replace("원본보기", '')
    txt = txt.strip()
    newss.loc[idx, 'text'] = txt
    idx += 1

newss.head()

,title,url,text,date,time,label
0,최지성 사장 “비전2020 구체화해 삼성전자 100년 기업으로”,https://m.news.naver.com/read.nhn?mode=LSD&mid...,올해 100년 기업을 향한 비전2020 을 구체화하기 위해 초경쟁력을 확보하라 ...,2010.01.04,2010-01-04 17:03:02,0.016069
1,"[CES 2010]삼성전자, 친환경 LED 대거 선보여",https://m.news.naver.com/read.nhn?mode=LSD&mid...,아시아경제 김정민 기자 삼성전자가 7일 미국 라스베이거스에서 열리는 세계 최대 멀티...,2010.01.05,2010-01-05 11:00:00,0.016069
2,"삼성전자 ""MP3플레이어도 보는 시대""",https://m.news.naver.com/read.nhn?mode=LSD&mid...,서울 연합뉴스 조성흠 기자 삼성전자는 오는 7일 미국 라스베이거스에서 열리는 ...,2010.01.05,2010-01-05 11:00:08,0.016069
3,"김연아-오서 코치, 삼성전자 에어컨 모델 발탁",https://n.news.naver.com/sports/general/articl...,마이데일리 김용우 기자 피겨 퀸 김연아 19 고려대 와 브라이언 오서 코치...,2010.01.05,2010-01-05 12:08:01,0.016069
4,삼성전자 사상 최고가…83만원 돌파,https://m.news.naver.com/read.nhn?mode=LSD&mid...,삼성전자 005930 가 개장과 함께 83만원을 넘어서며 사상최고가를 경신했다 6...,2010.01.06,2010-01-06 10:01:01,0.019592


In [10]:
newss['label'].astype(int)
newss['label'] = newss['label'].apply(lambda x: 1 if x >= 0 else 0)

newss.head()

,title,url,text,date,time,label
0,최지성 사장 “비전2020 구체화해 삼성전자 100년 기업으로”,https://m.news.naver.com/read.nhn?mode=LSD&mid...,올해 100년 기업을 향한 비전2020 을 구체화하기 위해 초경쟁력을 확보하라 ...,2010.01.04,2010-01-04 17:03:02,1
1,"[CES 2010]삼성전자, 친환경 LED 대거 선보여",https://m.news.naver.com/read.nhn?mode=LSD&mid...,아시아경제 김정민 기자 삼성전자가 7일 미국 라스베이거스에서 열리는 세계 최대 멀티...,2010.01.05,2010-01-05 11:00:00,1
2,"삼성전자 ""MP3플레이어도 보는 시대""",https://m.news.naver.com/read.nhn?mode=LSD&mid...,서울 연합뉴스 조성흠 기자 삼성전자는 오는 7일 미국 라스베이거스에서 열리는 ...,2010.01.05,2010-01-05 11:00:08,1
3,"김연아-오서 코치, 삼성전자 에어컨 모델 발탁",https://n.news.naver.com/sports/general/articl...,마이데일리 김용우 기자 피겨 퀸 김연아 19 고려대 와 브라이언 오서 코치...,2010.01.05,2010-01-05 12:08:01,1
4,삼성전자 사상 최고가…83만원 돌파,https://m.news.naver.com/read.nhn?mode=LSD&mid...,삼성전자 005930 가 개장과 함께 83만원을 넘어서며 사상최고가를 경신했다 6...,2010.01.06,2010-01-06 10:01:01,1


In [ ]:
newss.to_csv('data/labeled/samsung_2010_2021_01.csv')